<a href="https://colab.research.google.com/github/shuuuuyu/114-1-Programing-Language/blob/main/hw_2_%E6%88%90%E7%B8%BE%E4%B8%80%E6%9C%AC%E9%80%9A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### testing

In [5]:
# ============================================
# 成績管理系統 (Gradio + Gemini API + Google Sheets)
# ============================================

import pandas as pd
import gradio as gr
import numpy as np
import requests
import json
from google.colab import userdata, auth
from google.auth import default
import gspread

# ============================================
# 1. Google Sheets 授權與連接
# ============================================

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

SHEET_ID = "1imbM8DGLcECi-e1YoX-8Xt-kILgO1ehj1Lip1CdQ_uo"
sh = gc.open_by_key(SHEET_ID)

# ============================================
# 2. Gemini API 設定
# ============================================

def call_gemini(prompt_text):
    """呼叫 Gemini API"""
    api_key = userdata.get('gemini')
    url = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent"

    headers = {
        'Content-Type': 'application/json',
        'X-goog-api-key': api_key
    }

    data = {
        "contents": [{
            "parts": [{"text": prompt_text}]
        }]
    }

    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()
        result = response.json()

        if 'candidates' in result and len(result['candidates']) > 0:
            return result['candidates'][0]['content']['parts'][0]['text']
        else:
            return "未收到有效回應"
    except Exception as e:
        return f"API 呼叫錯誤: {e}"

# ============================================
# 3. 資料讀取函數
# ============================================

def load_grades_data():
    """從第一個分頁讀取成績資料"""
    worksheet = sh.sheet1
    data = worksheet.get_all_values()
    df = pd.DataFrame(data[1:], columns=data[0])

    # 轉換數值欄位
    numeric_cols = ['作業1', '作業2', '小考1', '小考2', '期中考', '期末考']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    return df

def load_open_answers():
    """從第二個分頁讀取開放式作答"""
    worksheet = sh.worksheets()[1]  # 第二個分頁
    data = worksheet.get_all_values()
    df = pd.DataFrame(data[1:], columns=data[0])
    return df

def get_student_list():
    """取得學生學號列表"""
    df = load_grades_data()
    return df['學號'].tolist()

# ============================================
# 4. 板塊一:成績計算與寫回
# ============================================

def show_student_grades(student_id):
    """顯示學生的各項成績"""
    df = load_grades_data()
    student = df[df['學號'] == student_id]

    if student.empty:
        return "查無此學生", "", "", "", "", "", ""

    student = student.iloc[0]
    return (
        student['姓名'],
        student['作業1'],
        student['作業2'],
        student['小考1'],
        student['小考2'],
        student['期中考'],
        student['期末考']
    )

def calculate_final_grade(student_id, hw_weight, quiz_weight, mid_weight, final_weight):
    """計算總成績"""
    # 檢查配比是否合理
    total_weight = hw_weight + quiz_weight + mid_weight + final_weight
    if abs(total_weight - 100) > 0.01:
        return f"❌ 配比總和必須為 100%,目前為 {total_weight}%,請重新設定!"

    df = load_grades_data()
    student = df[df['學號'] == student_id]

    if student.empty:
        return "查無此學生"

    student = student.iloc[0]

    # 計算各項平均
    hw_avg = (student['作業1'] + student['作業2']) / 2
    quiz_avg = (student['小考1'] + student['小考2']) / 2
    mid_score = student['期中考']
    final_score = student['期末考']

    # 計算總成績
    total = (hw_avg * hw_weight/100 +
             quiz_avg * quiz_weight/100 +
             mid_score * mid_weight/100 +
             final_score * final_weight/100)

    result = f"""
📊 **成績計算結果 - {student['姓名']} ({student_id})**

📝 作業平均: {hw_avg:.2f}
📋 小考平均: {quiz_avg:.2f}
📖 期中成績: {mid_score:.2f}
📘 期末成績: {final_score:.2f}

━━━━━━━━━━━━━━━━━━━━
🎯 **總成績: {total:.2f}**
━━━━━━━━━━━━━━━━━━━━

計算方式:
作業 {hw_weight}% + 小考 {quiz_weight}% + 期中 {mid_weight}% + 期末 {final_weight}%
    """

    return result

def write_back_to_sheet(student_id, hw_weight, quiz_weight, mid_weight, final_weight):
    """將計算結果寫回 Google Sheets"""
    # 檢查配比
    total_weight = hw_weight + quiz_weight + mid_weight + final_weight
    if abs(total_weight - 100) > 0.01:
        return f"❌ 配比錯誤,無法寫回!"

    try:
        df = load_grades_data()
        student_idx = df[df['學號'] == student_id].index[0]
        student = df.iloc[student_idx]

        # 計算成績
        hw_avg = (student['作業1'] + student['作業2']) / 2
        quiz_avg = (student['小考1'] + student['小考2']) / 2
        total = (hw_avg * hw_weight/100 +
                 quiz_avg * quiz_weight/100 +
                 student['期中考'] * mid_weight/100 +
                 student['期末考'] * final_weight/100)

        # 寫回 Sheet (假設在最後幾欄)
        worksheet = sh.sheet1
        row_num = student_idx + 2  # +2 因為有標題列且從1開始

        # 檢查是否有「總成績」欄位,沒有則新增
        headers = worksheet.row_values(1)
        if '作業平均' not in headers:
            worksheet.update_cell(1, len(headers)+1, '作業平均')
            worksheet.update_cell(1, len(headers)+2, '小考平均')
            worksheet.update_cell(1, len(headers)+3, '總成績')

        col_offset = len(headers) + 1
        worksheet.update_cell(row_num, col_offset, round(hw_avg, 2))
        worksheet.update_cell(row_num, col_offset+1, round(quiz_avg, 2))
        worksheet.update_cell(row_num, col_offset+2, round(total, 2))

        return f"✅ 成功將 {student['姓名']} 的成績寫回 Google Sheets!"

    except Exception as e:
        return f"❌ 寫回失敗: {e}"

# ============================================
# 5. 板塊二:開放式作答 AI 分析
# ============================================

def analyze_open_answers():
    """使用 Gemini 分析開放式作答"""
    try:
        df = load_open_answers()

        # 收集所有學生的回答
        all_answers = "\n".join([
            f"學生 {row['學號']} - {row['姓名']}: {row['開放式作答內容']}"
            for _, row in df.iterrows()
        ])

        # 設計提示詞
        prompt = f"""
你是一位專業的教育分析師。以下是學生們對於「歐洲氣候特徵及其對人類活動的影響」這個問題的開放式作答內容:

{all_answers}

請你分析這些回答,提供以下三個部分:

1. **整體摘要** (200字以內)
   - 總結學生們對歐洲氣候的整體理解程度
   - 指出多數學生掌握的核心概念

2. **常見迷思與錯誤觀念** (條列式,3-5點)
   - 找出學生回答中的誤解或不精確的概念
   - 說明正確的觀念應該是什麼

3. **教學建議** (3-4點)
   - 針對發現的問題,提供改進教學的具體建議

請用繁體中文,以清晰易讀的格式回答。
"""

        # 呼叫 Gemini
        analysis = call_gemini(prompt)

        return analysis

    except Exception as e:
        return f"分析失敗: {e}"

def save_analysis_to_sheet(analysis_text):
    """將分析結果存回第二個分頁"""
    try:
        worksheet = sh.worksheets()[1]

        # 在最後一欄寫入分析結果
        headers = worksheet.row_values(1)
        if 'AI分析結果' not in headers:
            worksheet.update_cell(1, len(headers)+1, 'AI分析結果')

        # 將分析結果寫在第二列開始
        col_num = len(headers) + 1
        worksheet.update_cell(2, col_num, analysis_text)

        return "✅ 分析結果已存回 Google Sheets 第二分頁!"

    except Exception as e:
        return f"❌ 存回失敗: {e}"

# ============================================
# 6. 板塊三:教學評量統計
# ============================================

def calculate_statistics(exam_type):
    """計算指定考試/作業的統計資料"""
    df = load_grades_data()

    if exam_type not in df.columns:
        return "查無此項目"

    scores = df[exam_type].dropna()

    mean_score = scores.mean()
    std_score = scores.std()
    max_score = scores.max()
    min_score = scores.min()
    median_score = scores.median()

    # 計算分數分布
    excellent = len(scores[scores >= 90])
    good = len(scores[(scores >= 80) & (scores < 90)])
    pass_count = len(scores[(scores >= 60) & (scores < 80)])
    fail = len(scores[scores < 60])

    result = f"""
📈 **{exam_type} - 統計分析報告**

━━━━━━━━━━━━━━━━━━━━
📊 基本統計量:
━━━━━━━━━━━━━━━━━━━━
• 平均分數: {mean_score:.2f}
• 標準差: {std_score:.2f}
• 中位數: {median_score:.2f}
• 最高分: {max_score:.2f}
• 最低分: {min_score:.2f}

━━━━━━━━━━━━━━━━━━━━
📋 分數分布:
━━━━━━━━━━━━━━━━━━━━
🌟 優秀 (90-100): {excellent} 人 ({excellent/len(scores)*100:.1f}%)
👍 良好 (80-89): {good} 人 ({good/len(scores)*100:.1f}%)
✓  及格 (60-79): {pass_count} 人 ({pass_count/len(scores)*100:.1f}%)
✗  不及格 (<60): {fail} 人 ({fail/len(scores)*100:.1f}%)

━━━━━━━━━━━━━━━━━━━━
💡 教學建議:
━━━━━━━━━━━━━━━━━━━━
"""

    # AI 生成教學建議
    if mean_score < 70:
        result += "• 整體平均偏低,建議加強基礎概念教學\n"
    if std_score > 15:
        result += "• 分數差異大,建議進行分組教學或個別輔導\n"
    if fail > len(scores) * 0.2:
        result += "• 不及格人數較多,建議重新檢視教材難度\n"

    return result

# ============================================
# 7. Gradio 介面設計
# ============================================

def create_interface():
    with gr.Blocks(title="成績管理系統", theme=gr.themes.Soft()) as demo:

        gr.Markdown("# 📚 智慧成績管理系統")
        gr.Markdown("整合 Google Sheets + Gemini AI 的成績分析平台")

        # ============ 板塊一:成績計算 ============
        with gr.Tab("📊 成績計算與管理"):
            gr.Markdown("## 學生成績查詢與計算")

            with gr.Row():
                student_dropdown = gr.Dropdown(
                    choices=get_student_list(),
                    label="選擇學生學號",
                    interactive=True
                )
                refresh_btn = gr.Button("🔄 重新載入學生名單", size="sm")

            with gr.Row():
                name_display = gr.Textbox(label="姓名", interactive=False)
                hw1 = gr.Textbox(label="作業1", interactive=False)
                hw2 = gr.Textbox(label="作業2", interactive=False)

            with gr.Row():
                quiz1 = gr.Textbox(label="小考1", interactive=False)
                quiz2 = gr.Textbox(label="小考2", interactive=False)
                mid = gr.Textbox(label="期中考", interactive=False)
                final = gr.Textbox(label="期末考", interactive=False)

            gr.Markdown("### 📐 設定成績配比")
            with gr.Row():
                hw_w = gr.Slider(0, 100, value=20, step=5, label="作業佔比 (%)")
                quiz_w = gr.Slider(0, 100, value=20, step=5, label="小考佔比 (%)")
                mid_w = gr.Slider(0, 100, value=30, step=5, label="期中考佔比 (%)")
                final_w = gr.Slider(0, 100, value=30, step=5, label="期末考佔比 (%)")

            with gr.Row():
                calc_btn = gr.Button("🧮 計算總成績", variant="primary")
                write_btn = gr.Button("💾 寫回 Google Sheets", variant="secondary")

            result_display = gr.Markdown()
            write_status = gr.Markdown()

            # 事件綁定
            refresh_btn.click(
                fn=lambda: gr.Dropdown(choices=get_student_list()),
                outputs=student_dropdown
            )

            student_dropdown.change(
                fn=show_student_grades,
                inputs=student_dropdown,
                outputs=[name_display, hw1, hw2, quiz1, quiz2, mid, final]
            )

            calc_btn.click(
                fn=calculate_final_grade,
                inputs=[student_dropdown, hw_w, quiz_w, mid_w, final_w],
                outputs=result_display
            )

            write_btn.click(
                fn=write_back_to_sheet,
                inputs=[student_dropdown, hw_w, quiz_w, mid_w, final_w],
                outputs=write_status
            )

        # ============ 板塊二:AI 分析 ============
        with gr.Tab("🤖 AI 開放式作答分析"):
            gr.Markdown("## 使用 Gemini 分析學生的開放式作答")
            gr.Markdown("將自動分析所有學生的回答,產生摘要、常見迷思與教學建議")

            analyze_btn = gr.Button("🔍 開始 AI 分析", variant="primary", size="lg")
            analysis_output = gr.Markdown(label="分析結果")

            save_analysis_btn = gr.Button("💾 存回 Google Sheets")
            save_status = gr.Markdown()

            analyze_btn.click(
                fn=analyze_open_answers,
                outputs=analysis_output
            )

            save_analysis_btn.click(
                fn=save_analysis_to_sheet,
                inputs=analysis_output,
                outputs=save_status
            )

        # ============ 板塊三:統計分析 ============
        with gr.Tab("📈 教學成效評量"):
            gr.Markdown("## 單項考試統計分析")

            exam_selector = gr.Radio(
                choices=['作業1', '作業2', '小考1', '小考2', '期中考', '期末考'],
                label="選擇要分析的項目",
                value='小考1'
            )

            stat_btn = gr.Button("📊 生成統計報告", variant="primary")
            stat_output = gr.Markdown()

            stat_btn.click(
                fn=calculate_statistics,
                inputs=exam_selector,
                outputs=stat_output
            )

        gr.Markdown("---")
        gr.Markdown("💡 **系統說明**: 本系統整合 Google Sheets 與 Gemini AI,提供成績管理、AI 分析與統計功能")

    return demo

# ============================================
# 8. 啟動應用
# ============================================

if __name__ == "__main__":
    app = create_interface()
    app.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d5864e4f52e73364ce.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d5864e4f52e73364ce.gradio.live
